In [4]:
import h5py
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import numpy as np
import sys
import time
import pandas as pd
import cv2
import os
import imageio
import matplotlib as mpl
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import shutil
import random

from deepcell.utils.tracking_utils import load_trks

/opt/conda/lib/python3.7/site-packages/tensorflow-2.5.0-py3.7-linux-x86_64.egg/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


# Make mask .npy

### Create variables

In [5]:
# Input file
fN = h5py.File("../22956814/22956814_featuresN.hdf5")

# Define dorsal and ventral coordinates
coordinates = fN['coordinates']
dorsal = coordinates['dorsal_contours']
ventral = coordinates['ventral_contours']

# trajectory_data and timeseries_data
traj = pd.read_hdf("../22956814/22956814_featuresN.hdf5", '/trajectories_data')
timeseries = fN['timeseries_data']

In [6]:
# Get wells - data from timeseries_data, at position [2]

wells = ['A1', 'A2', 'A3', 'A4', 'B1', 'B2', 'B3', 'B4', 'C1', 'C2', 'C3', 'C4', 'D1', 'D2', 'D3', 'D4']
#wells = [each.encode() for each in wells]

well_dict = {}
for each in wells:
    well_dict[each] = []
    
#print('A1'.encode().decode())
for i, each in enumerate(timeseries[:]):  # "i" == index
    well = each[2].decode()
    if well in well_dict.keys():
        well_dict[well].append(i)

### Functions

In [7]:
# Function - make dorsal ventral coordinates ==> d_x, d_y, v_x, v_y

def make_dorsal_ventral_coor(dorsal, ventral, skeleton_id):
    d_x = []
    d_y = []
    v_x = []
    v_y = []

    for each in dorsal[skeleton_id][:]:
        d_x.append(each[0]/12.4)
        d_y.append(each[1]/12.4)

    for each in ventral[skeleton_id][:]:
        v_x.append(each[0]/12.4)
        v_y.append(each[1]/12.4)
    
    return d_x, d_y, v_x, v_y

In [8]:
def make_mask(d_x, d_y, v_x, v_y, well, worm_index):
    polygon = []

    x_i = 0
    while x_i < len(d_x):
        polygon.append(d_x[x_i])
        polygon.append(d_y[x_i])
        x_i += 1

    y_i = len(v_x)-1
    while y_i >= 0:
        polygon.append(v_x[y_i])
        polygon.append(v_y[y_i])
        y_i -= 1

    width = 3600
    height = 3036

    img = Image.new('L', (width, height), 0)
    ImageDraw.Draw(img).polygon(polygon, outline=worm_index, fill=worm_index)
    
    # Define well coordinates
    if well == 'D4':
        mask = np.array(img)[180:710, 160:690]  
    elif well == 'D3':
        mask = np.array(img)[180:710, 880:1410]  
    elif well == 'D2':
        mask = np.array(img)[180:710, 1600:2130]
    elif well == 'D1':
        mask = np.array(img)[180:710, 2320:2850]
    elif well == 'C4':
        mask = np.array(img)[895:1425, 160:690]
    elif well == 'C3':
        mask = np.array(img)[895:1425, 880:1410]
    elif well == 'C2':
        mask = np.array(img)[895:1425, 1600:2130]
    elif well == 'C1':
        mask = np.array(img)[895:1425, 2320:2850]
    elif well == 'B4':
        mask = np.array(img)[1620:2150, 160:690] 
    elif well == 'B3':
        mask = np.array(img)[1620:2150, 880:1410] 
    elif well == 'B2':
        mask = np.array(img)[1620:2150, 1600:2130] 
    elif well == 'B1':
        mask = np.array(img)[1620:2150, 2320:2850] 
    elif well == 'A4':
        mask = np.array(img)[2340:2870, 180:710]
    elif well == 'A3':
        mask = np.array(img)[2340:2870, 880:1410]
    elif well == 'A2':
        mask = np.array(img)[2340:2870, 1600:2130]
    elif well == 'A1':
        mask = np.array(img)[2340:2870, 2320:2850] 
        
    mask_cp = np.copy(mask)
    mask_cp = mask_cp.astype(np.uint16)
    if worm_index != 255:
        mask_cp[mask_cp >= 255] = worm_index
        
    return mask_cp

In [16]:
# function - create 9002*530*530 mask for each well

def make_well_mask(well='A4', dorsal=dorsal, ventral=ventral, well_dict=well_dict, filename=False, down_sampling=1, frame_start=0, frame_end=9001, sequential=False):  
    pd = traj.loc[well_dict[well], :]  # So here it is the position_index, not skel_id
    list_stack = []
    start_time = time.time()
    array_2d = np.zeros(shape=(530, 530), dtype=np.int16)
    
    print("--------------------%s------------------------" % well)

    ## Second, do frame by frame
    frame_index = frame_start
    while frame_index <= frame_end:  
        array_2d = np.zeros(shape=(530, 530), dtype=np.int16)
        skel_id_inframe = list(pd[pd["frame_number"] == frame_index]['skeleton_id'])
        worm_id_inframe = list(pd[pd["frame_number"] == frame_index]['worm_index_joined'])

        for i, skel_id in enumerate(skel_id_inframe):
            worm_index = worm_id_inframe[i]
            dx, dy, vx, vy = make_dorsal_ventral_coor(dorsal, ventral, skel_id)
            mask = make_mask(dx, dy, vx, vy, well, worm_index)
            array_2d = array_2d + mask

        list_stack.append(array_2d)

        if frame_index == frame_start:
            print("Starts at: ", frame_index)
        
        if frame_index%200 == 0:
            print("Finish frame_index: ", frame_index)
            print("--- %s seconds ---" % (time.time() - start_time))

        if frame_index == frame_end:
            print("Finish!")
            print("--- %s seconds ---" % (time.time() - start_time))

        frame_index += 1
    print("length of list_stack: ", len(list_stack))
    stacked_array = np.stack(list_stack, axis=0)
    stacked_array = stacked_array[:,:,:, np.newaxis]
    
    if filename:
        output_filename = filename
    else:
        output_filename = "worm_data/22956814/mask_%s.npy" % well
    
    if down_sampling != 1:
        stacked_array = stacked_array[frame_start:(frame_end+1):down_sampling, :, :, :]
    
    # Added sequential here
    if sequential == True:
        # Check the unique values in the np arrays - the original worm indices
        uniques_original = list(np.unique(stacked_array))[1:]  # [1:] to omit values == '0'
        print('unique_worm_id: ', uniques_original)
        sequential_ids = range(1, len(uniques_original)+1)
        for i, original in enumerate(uniques_original):  
            stacked_array[stacked_array == uniques_original[i]] = sequential_ids[i]  # Update stacked_array with sequential worm_ids
    
    np.save(output_filename, stacked_array)
    
    return stacked_array

In [17]:
# Check that make_well_mask works when sequential == True

#a4_a = make_well_mask(well='A4', dorsal=dorsal, ventral=ventral, well_dict=well_dict, filename='../test/a4_a.npy', down_sampling=6, frame_start=0, frame_end=599, sequential=False)
a4_b = make_well_mask(well='A4', dorsal=dorsal, ventral=ventral, well_dict=well_dict, filename='../test/a4_b.npy', down_sampling=6, frame_start=0, frame_end=599, sequential=True)

--------------------A4------------------------
Starts at:  0
Finish frame_index:  0
--- 0.020763874053955078 seconds ---
Finish frame_index:  200
--- 3.1178090572357178 seconds ---
Finish frame_index:  400
--- 5.054049968719482 seconds ---
Finish!
--- 7.035767555236816 seconds ---
length of list_stack:  600
unique_worm_id:  [3]


### Create mask .npy

In [26]:
# Function to make all masks at once, with truncated videos. Each video starts at a new worm's first appearance

def make_well_masks(well, file_dir="../22956814/", sequential=False):
    pd= traj.loc[well_dict[well], :]
    worm_ids = pd['worm_index_joined']
    unique_worm_ids = list(np.unique(worm_ids)) 

    # Find the first frame for the above worm to appear:
    frame_starts = []
    for worm in unique_worm_ids:
        frame_starts.append(list(pd[pd['worm_index_joined'] == worm]['frame_number'])[0])

    # Also create random.randint between 1,10 for down_sampling:
    # random_down_sampling = [random.randint(1, 10) for x in range(len(unique_worm_ids))]
    random_down_sampling = [6 for x in range(len(unique_worm_ids))]
    
    not_empty = []
    rename_flag = 1

    for i, start in enumerate(frame_starts):
        frame_start = start
        down_sampling = random_down_sampling[i]
        frame_end = start + 100*down_sampling - 1
        fn = file_dir + str(well) + ("_mask_%d.npy" % (i+1))
            #fn = '../22956814/a4_mask_%d.npy' % (i+1)  # '+1' to avoid rewriting the next file!!!!

        mask_array = make_well_mask(well, down_sampling=down_sampling, frame_start=frame_start, frame_end=frame_end, filename=fn, sequential=sequential)
        if mask_array.size == 0:
            os.remove(fn)
        else:
            start_end_sampling = [frame_start, frame_end, down_sampling]  # For sampling raw video
            not_empty.append(start_end_sampling)
            if sequential == False:
                os.rename(fn, file_dir+str(well)+("_mask_%d.npy" % rename_flag))  # Rename the file so that 1,2....n
            elif sequential == True:
                os.rename(fn, file_dir+str(well)+("_mask_seq_%d.npy" % rename_flag))
            rename_flag += 1

    txt_fn = file_dir+str(well)+"_not_empty.txt"
    with open(txt_fn, 'w') as f:
        for each in not_empty[0:-1]:
            f.write(str(each[0]) + ',' + str(each[1]) + ',' + str(each[2]) + '\n')
        f.write(str(not_empty[-1][0]) + ',' + str(not_empty[-1][1]) + ',' + str(not_empty[-1][2]))

In [ ]:
for well in ['A1', 'A2', 'A3', 'A4', 'B1', 'B2', 'B3', 'B4', 'C1', 'C2', 'C3', 'C4', 'D1', 'D2', 'D3', 'D4']:
    make_well_masks(well)

In [28]:
# Test for sequential for make_well_masks

make_well_masks('A1')

--------------------A1------------------------
Starts at:  0
Finish frame_index:  0
--- 0.020069122314453125 seconds ---
Finish frame_index:  200
--- 2.742199420928955 seconds ---
Finish frame_index:  400
--- 5.2751195430755615 seconds ---
Finish!
--- 7.776747226715088 seconds ---
length of list_stack:  600
--------------------A1------------------------
Starts at:  2275
Finish frame_index:  2400
--- 1.553812026977539 seconds ---
Finish frame_index:  2600
--- 4.088731050491333 seconds ---
Finish frame_index:  2800
--- 6.24233078956604 seconds ---
Finish!
--- 6.312981605529785 seconds ---
length of list_stack:  600
--------------------A1------------------------
Starts at:  2650
Finish frame_index:  2800
--- 1.4699077606201172 seconds ---
Finish frame_index:  3000
--- 1.6569042205810547 seconds ---
Finish frame_index:  3200
--- 1.849391222000122 seconds ---
Finish!
--- 1.8957006931304932 seconds ---
length of list_stack:  600
--------------------A1------------------------
Starts at:  2651

In [29]:
for well in ['A1', 'A2', 'A3', 'A4', 'B1', 'B2', 'B3', 'B4', 'C1', 'C2', 'C3', 'C4', 'D1', 'D2', 'D3', 'D4']:
    make_well_masks(well, sequential=True)

--------------------A1------------------------
Starts at:  0
Finish frame_index:  0
--- 0.020926237106323242 seconds ---
Finish frame_index:  200
--- 2.6416311264038086 seconds ---
Finish frame_index:  400
--- 5.181337118148804 seconds ---
Finish!
--- 7.693621635437012 seconds ---
length of list_stack:  600
unique_worm_id:  [9]
--------------------A1------------------------
Starts at:  2275
Finish frame_index:  2400
--- 1.516096830368042 seconds ---
Finish frame_index:  2600
--- 3.9108121395111084 seconds ---
Finish frame_index:  2800
--- 5.926226377487183 seconds ---
Finish!
--- 5.997816801071167 seconds ---
length of list_stack:  600
unique_worm_id:  []
--------------------A1------------------------
Starts at:  2650
Finish frame_index:  2800
--- 1.4354145526885986 seconds ---
Finish frame_index:  3000
--- 1.6201531887054443 seconds ---
Finish frame_index:  3200
--- 1.8046932220458984 seconds ---
Finish!
--- 1.848642349243164 seconds ---
length of list_stack:  600
unique_worm_id:  []
